In [3]:
!pip install tensorflow tensorflow pandas matplotlib scikit-learn

In [35]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [36]:
df = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))

In [37]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


<b> Preprocess </b>

In [38]:
!pip list

Package                            Version
---------------------------------- -------------------
-illow                             10.0.0
absl-py                            1.4.0
aiofiles                           23.2.1
aiohttp                            3.8.5
aiosignal                          1.3.1
alabaster                          0.7.12
altair                             5.0.1
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
ansi2html                          1.8.0
anyio                              3.7.1
apikey                             0.2.4
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
astunparse                         1.6.3
async-generator                    1.10
async-timeout                      4.0.

In [39]:
from tensorflow.keras.layers import TextVectorization

certifi                            2020.12.5
cffi                               1.14.5
chardet                            4.0.0
charset-normalizer                 3.2.0
chess                              1.10.0
click                              8.1.7
click-default-group                1.2.2
cloudpickle                        1.6.0
clyent                             1.2.2
colorama                           0.4.4
coloredlogs                        15.0.1
comtypes                           1.1.9
conda                              4.10.1
conda-build                        3.21.4
conda-content-trust                0+unknown
conda-package-handling             1.7.3
conda-repo-cli                     1.0.4
conda-token                        0.3.0
conda-verify                       3.4.2
contextlib2                        0.6.0.post1
cryptography                       3.4.7
cycler                             0.10.0
Cython                             0.29.23
cytoolz                            

In [40]:
X = df['comment_text']
y = df[df.columns[2:]].values


jupyter-server                     1.4.1
jupyterlab                         3.0.14
jupyterlab-pygments                0.1.2
jupyterlab-server                  2.4.0
jupyterlab-widgets                 1.0.0
keras                              2.12.0
keyring                            22.3.0
kiwisolver                         1.3.1
langchain                          0.0.264
langsmith                          0.0.22
lazy-object-proxy                  1.6.0
libarchive-c                       2.9
libclang                           16.0.0
llvmlite                           0.36.0
locket                             0.2.1
lxml                               4.6.3
Markdown                           3.4.3
markdown-it-py                     3.0.0
MarkupSafe                         2.1.3
marshmallow                        3.20.1
matplotlib                         3.3.4
mccabe                             0.6.1
mdurl                              0.1.2
menuinst                           1.4.16
mistune

In [41]:
MAX_FEATURES = 200000 # number of words in the vocab


pluggy                             0.13.1
ply                                3.11
pooch                              1.7.0
prometheus-client                  0.10.1
prompt-toolkit                     3.0.17
protobuf                           4.22.1
psutil                             5.8.0
ptyprocess                         0.7.0
py                                 1.10.0
pyarrow                            12.0.1
pyasn1                             0.4.8
pyasn1-modules                     0.2.8
PyAutoGUI                          0.9.53
pyclipper                          1.3.0.post4
pycodestyle                        2.6.0
pycosat                            0.6.3
pycparser                          2.20
pycurl                             7.43.0.6
pydantic                           1.10.12
pydeck                             0.8.0
pydocstyle                         6.0.0
pydub                              0.25.1
pyerfa                             1.7.3
pyflakes                           2.2.

In [42]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [43]:
vectorizer.adapt(X.values)

In [44]:
vectorized_text = vectorizer(X.values)

In [45]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [46]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

<b> 2. Create Sequential Model </b>

In [48]:
 from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [49]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [50]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [25]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 14914s 2s/step - loss: 0.0629 - val_loss: 0.0470


In [52]:
from matplotlib import pyplot as plt

In [53]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

NameError: name 'history' is not defined

<Figure size 576x360 with 0 Axes>

<b>3. Make Predictions </b>

In [47]:
input_text = vectorizer(['You freaking suck! I am going to hit you.'])
res = model.predict(input_text)


AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'predict'

In [32]:
(res > 0.5).astype(int)

array([[1, 0, 1, 0, 1, 0]])

In [54]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [55]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 12s 12s/step


array([[0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0]])

In [35]:
res.shape

(1, 6)

In [36]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [37]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [38]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 392ms/step


In [39]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8462410569190979, Recall:0.6278026700019836, Accuracy:0.46439316868782043


<b>5. Test and Gradio </b>

In [52]:
model.save('toxicity.h5')

In [53]:
model = tf.keras.models.load_model('toxicity.h5')

In [54]:
input_str = vectorizer('hey i freaken hate you!')

In [55]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 2s 2s/step


In [56]:
res

array([[0.8090163 , 0.03164391, 0.39825737, 0.02182293, 0.37150934,
        0.0729809 ]], dtype=float32)

In [30]:
import tensorflow as tf
import gradio as gr

In [31]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

<ipython-input-31-f5d8ac15336b>:2: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
<ipython-input-31-f5d8ac15336b>:2: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
<ipython-input-31-f5d8ac15336b>:2: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),


In [64]:
interface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on public URL: https://f34ccc18ea7fc3e23a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 1s 1s/step
